In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import os
import gc
import warnings
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import RobustScaler
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
import lightgbm as lgb
import seaborn as sns
import math
import matplotlib.pylab as plt
# from meanencoder import MeanEncoder
import xgboost as xgb
import geohash2 as gh
from math import *

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
#读数据
path='data'
op_train = pd.read_csv(path + '/operation_train_new.csv',dtype={'os':str,'success':np.float64})
trans_train = pd.read_csv(path + '/transaction_train_new.csv',dtype={'channel':str,'trans_type2':str,'market_type':str})

op_test = pd.read_csv(path + '/operation_round1_new.csv',dtype={'os':str,'success':np.float64})
trans_test = pd.read_csv(path + '/transaction_round1_new.csv',dtype={'channel':str,'trans_type2':str,'market_type':str})
y = pd.read_csv(path + '/tag_train_new.csv')
sub = pd.read_csv(path + '/sub.csv')
print('load data ok...')

load data ok...


In [1]:
# op_train.info()
# trans_train.info()

In [ ]:
# 数据预处理

# 计算星期 week_day
op_train['week_day']=op_train['day'].apply(lambda x:(x+5)%7)
op_test['week_day']=op_test['day'].apply(lambda x:x%7)

# 计算hour
op_train['hour']=pd.to_datetime(op_train['time']).dt.hour #时
op_test['hour']=pd.to_datetime(op_test['time']).dt.hour #时

# 计算minute
op_train["minute"]=pd.to_datetime(op_train['time']).dt.minute # 分
op_test["minute"]=pd.to_datetime(op_test['time']).dt.minute # 分
                                         
# 计算second
op_train["second"]=pd.to_datetime(op_train['time']).dt.second # 秒
op_test["second"]=pd.to_datetime(op_test['time']).dt.second # 秒
                                         
op_train['int_time']=op_train['hour']*60*60+op_train['minute']*60+op_train['second']
op_test['int_time']=op_test['hour']*60*60+op_test['minute']*60+op_test['second']

# op_train_y
op_train_y=op_train.merge(y,on='UID',how='left')

In [ ]:
op_train.head()

In [ ]:
# 计算星期 week_day
trans_train['week_day']=trans_train['day'].apply(lambda x:(x+5)%7)
trans_test['week_day']=trans_test['day'].apply(lambda x:x%7)

# 计算hour
trans_train['hour']=pd.to_datetime(trans_train['time']).dt.hour #时
trans_test['hour']=pd.to_datetime(trans_test['time']).dt.hour #时

# 计算minute
trans_train["minute"]=pd.to_datetime(trans_train['time']).dt.minute # 分
trans_test["minute"]=pd.to_datetime(trans_test['time']).dt.minute # 分
                                         
# 计算second
trans_train["second"]=pd.to_datetime(trans_train['time']).dt.second # 秒
trans_test["second"]=pd.to_datetime(trans_test['time']).dt.second # 秒
                                         
trans_train['int_time']=trans_train['hour']*60*60+trans_train['minute']*60+trans_train['second']
trans_test['int_time']=trans_test['hour']*60*60+trans_test['minute']*60+trans_test['second']

# 连续值排序
trans_train['trans_amt_rank'] = trans_train['trans_amt'].rank(method='max')
trans_train['bal_rank'] = trans_train['bal'].rank(method='max')

trans_test['trans_amt_rank'] = trans_test['trans_amt'].rank(method='max')
trans_test['bal_rank'] = trans_test['bal'].rank(method='max')

# op_train_y
trans_train_y=trans_train.merge(y,on='UID',how='left')

In [ ]:
# def decode_geohash_longtitude(geo_code):
#     if geo_code=='':
#         return np.nan
#     else:
#         longtitude,latitude=gh.decode(geo_code)
#         return int(longtitude)

# def decode_geohash_latitude(geo_code):
#     if geo_code=='':
#         return np.nan
#     else:
#         longtitude,latitude=gh.decode(geo_code)
#         return int(latitude)

# op_train['longtitude']=op_train['geo_code'].fillna('').apply(lambda x:decode_geohash_longtitude(x))
# op_train['latitude']=op_train['geo_code'].fillna('').apply(lambda x:decode_geohash_latitude(x))

# op_test['longtitude']=op_test['geo_code'].fillna('').apply(lambda x:decode_geohash_longtitude(x))
# op_test['latitude']=op_test['geo_code'].fillna('').apply(lambda x:decode_geohash_latitude(x))

# trans_train['longtitude']=trans_train['geo_code'].fillna('').apply(lambda x:decode_geohash_longtitude(x))
# trans_train['latitude']=trans_train['geo_code'].fillna('').apply(lambda x:decode_geohash_latitude(x))

# trans_test['longtitude']=trans_test['geo_code'].fillna('').apply(lambda x:decode_geohash_longtitude(x))
# trans_test['latitude']=trans_test['geo_code'].fillna('').apply(lambda x:decode_geohash_latitude(x))

In [ ]:
# trans_train.drop(['trans_amt_rank_discret'],axis=1,inplace=True)

In [ ]:
# trans_train['trans_amt_rank_discret']=0
# trans_max=trans_train['trans_amt_rank'].max()
# trans_min=trans_train['trans_amt_rank'].min()
# avg=np.round((trans_max-trans_min)/10)
# for i in range(10):
#     up=trans_min+(i+1)*avg
#     down=trans_min+i*avg
#     index=trans_train[(trans_train['trans_amt_rank']<up)&(trans_train['trans_amt_rank']>=down)].index
#     trans_train.trans_amt_rank_discret[index]=i+1

# trans_train['bal_rank_discret']=0
# trans_max=trans_train['bal_rank'].max()
# trans_min=trans_train['bal_rank'].min()
# avg=np.round((trans_max-trans_min)/10)
# for i in range(10):
#     up=trans_min+(i+1)*avg
#     down=trans_min+i*avg
#     index=trans_train[(trans_train['bal_rank']<up)&(trans_train['bal_rank']>=down)].index
#     trans_train.bal_rank_discret[index]=i+1

In [ ]:
# trans_test['trans_amt_rank_discret']=0
# trans_max=trans_test['trans_amt_rank'].max()
# trans_min=trans_test['trans_amt_rank'].min()
# avg=np.round((trans_max-trans_min)/10)
# for i in range(10):
#     up=trans_min+(i+1)*avg
#     down=trans_min+i*avg
#     index=trans_test[(trans_test['trans_amt_rank']<up)&(trans_test['trans_amt_rank']>=down)].index
#     trans_test.trans_amt_rank_discret[index]=i+1

# trans_test['bal_rank_discret']=0
# trans_max=trans_test['bal_rank'].max()
# trans_min=trans_test['bal_rank'].min()
# avg=np.round((trans_max-trans_min)/10)
# for i in range(10):
#     up=trans_min+(i+1)*avg
#     down=trans_min+i*avg
#     index=trans_test[(trans_test['bal_rank']<up)&(trans_test['bal_rank']>=down)].index
#     trans_test.bal_rank_discret[index]=i+1

In [ ]:
trans_train['log_trans_amt']=round(np.log(trans_train['trans_amt']),2)
trans_test['log_trans_amt']=round(np.log(trans_test['trans_amt']),2)

trans_train['log_bal']=round(np.log(trans_train['bal']),2)
trans_test['log_bal']=round(np.log(trans_test['bal']),2)

In [1]:
pd.set_option('display.max_columns', None)
# trans_train1[trans_train1.trans_amt_rank_discret==0]
# op_train_y[op_train_y.Tag==1].tail()
op_train.head()

NameError: name 'pd' is not defined

In [106]:
###### 处理op_train的数据
# 先查看操作箱单数据表
train=y
op_columns=['UID','Tag']

# 计算用户在30天内的总操作数
op_columns.append('op_operation_count')
train=train.merge(op_train.groupby(['UID'])['day'].count().reset_index(),on='UID',how='left')
# 操作日期 day
# 用户在30天内的活跃天数
op_columns.append('op_day_nunique')
train=train.merge(op_train.groupby(['UID'])['day'].nunique().reset_index(),on='UID',how='left')
# 用户在周末的操作次数
op_columns.append('op_operation_weekend_count')
train=train.merge(op_train[(op_train.week_day==0)|(op_train.week_day==6)].groupby(['UID'])['day'].count().reset_index(),on='UID',how='left')

# 用户有多少不同的操作类型
op_columns.append('op_mode_nunique')
train=train.merge(op_train.groupby(['UID'])['mode'].nunique().reset_index(),on='UID',how='left')

# 排名前5的操作类型
# c8741ce15ceac2a4,d25caee90b27fa9b,79f8b86938180c3c,b501fa4fc58206b9,acfaded7e04e7ba0
mode_top5=['c8741ce15ceac2a4','d25caee90b27fa9b','79f8b86938180c3c','b501fa4fc58206b9','acfaded7e04e7ba0']
for mode in mode_top5:
    op_columns.append('op_mode_'+mode+'_count')
    train=train.merge(op_train[op_train['mode']==mode].groupby(['UID'])['mode'].count().reset_index(),on='UID',how='left')

# 操作success的数量/总操作数
op_columns.append('op_success_mean')
train=train.merge(op_train.groupby(['UID'])['success'].mean().reset_index(),on='UID',how='left')

# 排名前5的hour和羊毛党操作最多的hour
# 8,9,10,11,14,15,16
hour_top5=[8,9,10,11,14,15,16]
for hour in hour_top5:
    op_columns.append('op_hour_'+str(hour)+'_count')
    train=train.merge(op_train[op_train['hour']==hour].groupby(['UID'])['hour'].count().reset_index(),on='UID',how='left')

# 不同的操作系统数量
op_columns.append('op_os_nunique')
train=train.merge(op_train.groupby(['UID'])['os'].nunique().reset_index(),on='UID',how='left')
# 排名前3的操作系统
os_top3=['102','200','103']
for os in os_top3:
    op_columns.append('op_os_'+os+'_count')
    train=train.merge(op_train[op_train['os']==os].groupby(['UID'])['os'].count().reset_index(),on='UID',how='left')

# 不同version的操作数量
op_columns.append('op_version_nunique')
train=train.merge(op_train.groupby(['UID'])['version'].nunique().reset_index(),on='UID',how='left')
# 排名前5的version
version_top5=['7.0.9','7.0.5','7.0.2','7.0.7','6.6.2','7.0.0']
for version in version_top5:
    op_columns.append('op_version_'+version+'_count')
    train=train.merge(op_train[op_train['version']==version].groupby(['UID'])['version'].count().reset_index(),on='UID',how='left')

# device1
op_columns.append('op_device1_nunique')
train=train.merge(op_train.groupby(['UID'])['device1'].nunique().reset_index(),on='UID',how='left')
device1_UID_num=op_train[['UID','device1']].drop_duplicates().groupby(['device1'])['UID'].count().sort_values(ascending=False)[0:10].index.tolist()
device1_top5=['09baf2f39bc3dc86', 'aca4977fbe8741e1', 'd2cf44cec09806cc', '49dd36968dbfadda', '8399be6b80847240', '630a1adff2a87007']
device1_top=set(device1_UID_num)|set(device1_top5)
for device1 in device1_top:
    op_columns.append('op_device1_'+device1+'_count')
    train=train.merge(op_train[op_train['device1']==device1].groupby(['UID'])['device1'].count().reset_index(),on='UID',how='left')

# 计算属性值和TopN的计数值
cols=['device_code2', 'device_code3', 'mac1','mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub']
for col in cols:
    op_columns.append('op_'+col+'_nunique')
    train=train.merge(op_train.groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
    index1=op_train.groupby([col])['UID'].count().sort_values(ascending=False)[0:5].index
    index2=op_train_y[op_train_y.Tag==1][col].value_counts()[0:5].index
    indexs=set(index1)|set(index2)
    
#     UID_num=op_train[['UID',col]].drop_duplicates().groupby([col])['UID'].count().sort_values(ascending=False)[0:10].index.tolist()
#     indexs=set(indexs)|set(UID_num)
    for index in index2:
        op_columns.append('op_'+col+'_'+index+'_count')
        train=train.merge(op_train[op_train[col]==index].groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
        

# # longtitude
# col='longtitude'
# op_columns.append('op_'+col+'_count')
# op_columns.append('op_'+col+'_nunique')
# op_columns.append('op_'+col+'_max')
# op_columns.append('op_'+col+'_min')
# op_columns.append('op_'+col+'_sum')
# op_columns.append('op_'+col+'_mean')
# op_columns.append('op_'+col+'_median')
# op_columns.append('op_'+col+'_std')
# op_columns.append('op_'+col+'_skew')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].max().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].min().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].sum().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].mean().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].median().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].std().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].skew().reset_index(),on='UID',how='left')

# # latitude
# col='latitude'
# op_columns.append('op_'+col+'_count')
# op_columns.append('op_'+col+'_nunique')
# op_columns.append('op_'+col+'_max')
# op_columns.append('op_'+col+'_min')
# op_columns.append('op_'+col+'_sum')
# op_columns.append('op_'+col+'_mean')
# op_columns.append('op_'+col+'_median')
# op_columns.append('op_'+col+'_std')
# op_columns.append('op_'+col+'_skew')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].max().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].min().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].sum().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].mean().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].median().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].std().reset_index(),on='UID',how='left')
# train =train.merge(op_train[op_train[col]!=0].groupby(['UID'])[col].skew().reset_index(),on='UID',how='left')

# 二元组合变量
bivariable=[['device1','device2'],['device1','mac1'],['device_code1','device_code2'],['device_code1','geo_code'],['device_code1','ip1']]
for var in bivariable:
    var1=var[0]
    var2=var[1]
    biparam=op_train_y[op_train_y.Tag==1].groupby([var1,var2])['UID'].count().sort_values(ascending=False).reset_index()[0:10]
    for param1,param2 in zip(biparam[var1],biparam[var2]):
        op_columns.append('op_'+var1+'_'+str(param1)+'_'+var2+'_'+str(param2)+'_count')
        train=train.merge(op_train[(op_train[var1]==param1)&(op_train[var2]==param2)].groupby(['UID']).size().reset_index(),on='UID',how='left')

In [107]:
op_train_int_time=op_train.groupby(['UID','day'])['int_time'].std().reset_index()
op_train_int_time_std_min=op_train_int_time[op_train_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].min().reset_index()
op_train_int_time_std_max=op_train_int_time[op_train_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].max().reset_index()
op_columns.append('op_'+'int_time'+'_std_min')
op_columns.append('op_'+'int_time'+'_std_max')
op_columns.append('op_'+'int_time'+'_std_mean')
op_columns.append('op_'+'int_time'+'_std_sum')
train=train.merge(op_train_int_time_std_min,on='UID',how='left')
train=train.merge(op_train_int_time_std_max,on='UID',how='left')
train=train.merge(op_train_int_time[op_train_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].mean().reset_index(),on='UID',how='left')
train=train.merge(op_train_int_time[op_train_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].sum().reset_index(),on='UID',how='left')

In [108]:
# 经纬度数据的处理

# input Lat_A 纬度A
# input Lng_A 经度A
# input Lat_B 纬度B
# input Lng_B 经度B
# output distance 距离(km)
# def calcDistance(Lat_A, Lng_A, Lat_B, Lng_B):
#     ra = 6378.140  # 赤道半径 (km)
#     rb = 6356.755  # 极半径 (km)
#     flatten = (ra - rb) / ra  # 地球扁率
#     rad_lat_A = radians(Lat_A)
#     rad_lng_A = radians(Lng_A)
#     rad_lat_B = radians(Lat_B)
#     rad_lng_B = radians(Lng_B)
#     pA = atan(rb / ra * tan(rad_lat_A))
#     pB = atan(rb / ra * tan(rad_lat_B))
#     xx = acos(sin(pA) * sin(pB) + cos(pA) * cos(pB) * cos(rad_lng_A - rad_lng_B))
#     c1 = (sin(xx) - xx) * (sin(pA) + sin(pB)) ** 2 / cos(xx / 2) ** 2
#     c2 = (sin(xx) + xx) * (sin(pA) - sin(pB)) ** 2 / sin(xx / 2) ** 2
#     dr = flatten / 8 * (c1 - c2)
#     distance = ra * (xx + dr)
#     return distance

# geo_code_list=op_train[['UID','longtitude','latitude']].dropna().drop_duplicates()
# UID_list=geo_code_list.UID.drop_duplicates()
# df = pd.DataFrame()
# for uid in UID_list:
#     longtitude=geo_code_list[geo_code_list.UID==uid]['longtitude'].tolist()
#     latitude=geo_code_list[geo_code_list.UID==uid]['latitude'].tolist()
#     count=len(longtitude)
#     dis_mean=0
#     dis_list=[]
#     if count==1:
#         dis_mean=0
#     else:
#         for i in range(count-1):
#             lo=longtitude[i]
#             la=latitude[i]
#             #print(lo,la)
#             dis=[calcDistance(la,lo,latitude[j],longtitude[j]) for j in range(i+1,count)]
#             dis_list.extend(dis)
#         dis_mean=np.mean(dis_list)
#         #print(dis_mean)
#     df2=pd.DataFrame({'UID':uid, 'distance_mean':dis_mean},index=["0"])
#     df=df.append(df2, ignore_index=True)
# op_columns.append('op_geo_code_distance_mean')
# train=train.merge(df,on='UID',how='left')

In [109]:
# 处理trans_train的数据
# 计算用户在30天内的总操作数
op_columns.append('trans_operation_count')
train=train.merge(trans_train.groupby(['UID'])['day'].count().reset_index(),on='UID',how='left')

# 用户在周末的操作次数
op_columns.append('trans_operation_weekend_count')
train=train.merge(trans_train[(trans_train.week_day==0)|(trans_train.week_day==6)].groupby(['UID'])['day'].count().reset_index(),on='UID',how='left')


# 计算属性值和TopN的计数值
cols=['channel', 'day', 'amt_src1', 'merchant',
       'code1', 'code2', 'trans_type1', 'acc_id1', 'device_code1',
       'device_code2', 'device_code3', 'device1', 'device2', 'mac1', 'ip1',
       'amt_src2', 'acc_id2', 'acc_id3', 'geo_code', 'trans_type2',
       'market_code', 'market_type', 'ip1_sub', 'hour']
for col in cols:
    op_columns.append('trans_'+str(col)+'_nunique')
    train=train.merge(trans_train.groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
    index1=trans_train.groupby([col])['UID'].count().sort_values(ascending=False)[0:5].index
    index2=trans_train_y[trans_train_y.Tag==1][col].value_counts()[0:5].index
    indexs=set(index1)|set(index2)
    for index in index2:
        op_columns.append('trans_'+str(col)+'_'+str(index)+'_count')
        train=train.merge(trans_train[trans_train[col]==index].groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
        
# trans_amt_rank_discret/bal_rank_discret
# rank=[1,2,3,4,5,6,7,8,9,10]
# for r in rank:
#     op_columns.append('trans_trans_amt_rank_discret'+'_'+str(r)+'_count')
#     train=train.merge(trans_train[trans_train['trans_amt_rank_discret']==r].groupby(['UID'])['trans_amt_rank_discret'].count().reset_index(),on='UID',how='left')

# for r in rank:
#     op_columns.append('trans_bal_rank_discret'+'_'+str(r)+'_count')
#     train=train.merge(trans_train[trans_train['bal_rank_discret']==r].groupby(['UID'])['bal_rank_discret'].count().reset_index(),on='UID',how='left')

col='trans_amt'
op_columns.append('trans_'+col+'_count')
op_columns.append('trans_'+col+'_nunique')
op_columns.append('trans_'+col+'_max')
op_columns.append('trans_'+col+'_min')
op_columns.append('trans_'+col+'_sum')
op_columns.append('trans_'+col+'_mean')
op_columns.append('trans_'+col+'_median')
op_columns.append('trans_'+col+'_std')
op_columns.append('trans_'+col+'_skew')
train =train.merge(trans_train.groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].max().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].min().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].sum().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].mean().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].median().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].std().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].skew().reset_index(),on='UID',how='left')

col='bal'
op_columns.append('trans_'+col+'_count')
op_columns.append('trans_'+col+'_nunique')
op_columns.append('trans_'+col+'_max')
op_columns.append('trans_'+col+'_min')
op_columns.append('trans_'+col+'_sum')
op_columns.append('trans_'+col+'_mean')
op_columns.append('trans_'+col+'_median')
op_columns.append('trans_'+col+'_std')
op_columns.append('trans_'+col+'_skew')
train =train.merge(trans_train.groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].max().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].min().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].sum().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].mean().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].median().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].std().reset_index(),on='UID',how='left')
train =train.merge(trans_train.groupby(['UID'])[col].skew().reset_index(),on='UID',how='left')

# 处理交易金额
op_columns.append('trans_'+'log_trans_amt_lower_5'+'_count')
train=train.merge(trans_train[trans_train['log_trans_amt']<5].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_5_to_6'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=5)&(trans_train['log_trans_amt']<6)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_6_to_7'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=6)&(trans_train['log_trans_amt']<7)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_7_to_8'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=7)&(trans_train['log_trans_amt']<8)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_8_to_9'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=8)&(trans_train['log_trans_amt']<9)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_9_to_10'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=9)&(trans_train['log_trans_amt']<10)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_10_to_11'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=10)&(trans_train['log_trans_amt']<11)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_11_to_12'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=11)&(trans_train['log_trans_amt']<12)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_12_to_13'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=12)&(trans_train['log_trans_amt']<13)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_13_to_14'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=13)&(trans_train['log_trans_amt']<14)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_14_to_15'+'_count')
train=train.merge(trans_train[(trans_train['log_trans_amt']>=14)&(trans_train['log_trans_amt']<=15)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_greater_15'+'_count')
train=train.merge(trans_train[trans_train['log_trans_amt']>15].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

In [110]:
# geo_code_list=trans_train[['UID','longtitude','latitude']].dropna().drop_duplicates()
# UID_list=geo_code_list.UID.drop_duplicates()
# df = pd.DataFrame()
# for uid in UID_list:
#     longtitude=geo_code_list[geo_code_list.UID==uid]['longtitude'].tolist()
#     latitude=geo_code_list[geo_code_list.UID==uid]['latitude'].tolist()
#     count=len(longtitude)
#     dis_mean=0
#     dis_list=[]
#     if count==1:
#         dis_mean=0
#     else:
#         for i in range(count-1):
#             lo=longtitude[i]
#             la=latitude[i]
#             #print(lo,la)
#             dis=[calcDistance(la,lo,latitude[j],longtitude[j]) for j in range(i+1,count)]
#             dis_list.extend(dis)
#         dis_mean=np.mean(dis_list)
#         #print(dis_mean)
#     df2=pd.DataFrame({'UID':uid, 'distance_mean':dis_mean},index=["0"])
#     df=df.append(df2, ignore_index=True)
# op_columns.append('trans_geo_code_distance_mean')
# train=train.merge(df,on='UID',how='left')

In [111]:
trans_train_int_time=trans_train.groupby(['UID','day'])['int_time'].std().reset_index()
trans_train_int_time_std_min=trans_train_int_time[trans_train_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].min().reset_index()
trans_train_int_time_std_max=trans_train_int_time[trans_train_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].max().reset_index()
op_columns.append('trans_'+'int_time'+'_std_min')
op_columns.append('trans_'+'int_time'+'_std_max')
op_columns.append('trans_'+'int_time'+'_std_mean')
op_columns.append('trans_'+'int_time'+'_std_sum')
train=train.merge(trans_train_int_time_std_min,on='UID',how='left')
train=train.merge(trans_train_int_time_std_max,on='UID',how='left')
train=train.merge(trans_train_int_time[trans_train_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].mean().reset_index(),on='UID',how='left')
train=train.merge(trans_train_int_time[trans_train_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].sum().reset_index(),on='UID',how='left')

In [112]:
###### 修改列名
train.columns=op_columns

# 用户操作总数/活跃天数
train['op_operation_per_day']=train['op_operation_count']/train['op_day_nunique']
# 用户周末操作数/总操作数
train['op_operation_per_weekend']=train['op_operation_weekend_count']/train['op_operation_count']

# 用户交易总数/活跃天数
train['trans_operation_per_day']=train['trans_operation_count']/train['trans_day_nunique']
# 用户周末交易数/总交易数
train['trans_operation_per_weekend']=train['trans_operation_weekend_count']/train['trans_operation_count']

#用户交易数/操作数
train['trans_per_op']=train['trans_operation_count']/train['op_operation_count']
# 用户操作数/交易数
train['op_per_trans']=train['op_operation_count']/train['trans_operation_count']

# 交易金额小于5的操作百分比
train['trans_log_trans_amt_lower_5_percent']=train['trans_log_trans_amt_lower_5_count']/train['trans_operation_count']

# 交易金额5-6的操作百分比
train['trans_log_trans_amt_5_to_6_percent']=train['trans_log_trans_amt_5_to_6_count']/train['trans_operation_count']

# 交易金额6-7的操作百分比
train['trans_log_trans_amt_6_to_7_percent']=train['trans_log_trans_amt_6_to_7_count']/train['trans_operation_count']
train['trans_log_trans_amt_7_to_8_percent']=train['trans_log_trans_amt_7_to_8_count']/train['trans_operation_count']
train['trans_log_trans_amt_8_to_9_percent']=train['trans_log_trans_amt_8_to_9_count']/train['trans_operation_count']
train['trans_log_trans_amt_9_to_10_percent']=train['trans_log_trans_amt_9_to_10_count']/train['trans_operation_count']
train['trans_log_trans_amt_10_to_11_percent']=train['trans_log_trans_amt_10_to_11_count']/train['trans_operation_count']
train['trans_log_trans_amt_11_to_12_percent']=train['trans_log_trans_amt_11_to_12_count']/train['trans_operation_count']
train['trans_log_trans_amt_12_to_13_percent']=train['trans_log_trans_amt_12_to_13_count']/train['trans_operation_count']
train['trans_log_trans_amt_13_to_14_percent']=train['trans_log_trans_amt_13_to_14_count']/train['trans_operation_count']
train['trans_log_trans_amt_14_to_15_percent']=train['trans_log_trans_amt_14_to_15_count']/train['trans_operation_count']
# 交易金额大于15的操作百分比
train['trans_log_trans_amt_greater_15_percent']=train['trans_log_trans_amt_greater_15_count']/train['trans_operation_count']

In [113]:
pd.set_option('display.max_columns', None)
train.head()

,UID,Tag,op_operation_count,op_day_nunique,op_operation_weekend_count,op_mode_nunique,op_mode_c8741ce15ceac2a4_count,op_mode_d25caee90b27fa9b_count,op_mode_79f8b86938180c3c_count,op_mode_b501fa4fc58206b9_count,op_mode_acfaded7e04e7ba0_count,op_success_mean,op_hour_8_count,op_hour_9_count,op_hour_10_count,op_hour_11_count,op_hour_14_count,op_hour_15_count,op_hour_16_count,op_os_nunique,op_os_102_count,op_os_200_count,op_os_103_count,op_version_nunique,op_version_7.0.9_count,op_version_7.0.5_count,op_version_7.0.2_count,op_version_7.0.7_count,op_version_6.6.2_count,op_version_7.0.0_count,op_device1_nunique,op_device1_630a1adff2a87007_count,op_device1_49dd36968dbfadda_count,op_device1_09baf2f39bc3dc86_count,op_device1_dcc26c88f36170cc_count,op_device1_8399be6b80847240_count,op_device1_0d4e77d48255a0a2_count,op_device1_bf872c181a38310a_count,op_device1_a92470e4f878a3f4_count,op_device1_aca4977fbe8741e1_count,op_device1_8f970961c66d9ec6_count,op_device1_d2cf44cec09806cc_count,op_device_code2_nunique,op_device_code2_483ddc3a31666b8f_count,op_device_code2_c3126d3ef88b84d0_count,op_device_code2_09b787daa79cead5_count,op_device_code2_59dcb10df09acd30_count,op_device_code2_83cd44a2841ceecc_count,op_device_code3_nunique,op_device_code3_42100099fcace21b_count,op_device_code3_a2ae68874761cf91_count,op_device_code3_0d1884730db15ddc_count,op_device_code3_d059c94c8b9fbfb9_count,op_device_code3_27fe4b8e9696e8ae_count,op_mac1_nunique,op_mac1_a8dc52f65085212e_count,op_mac1_b1f92d4324c1ee16_count,op_mac1_9251892b01b1a219_count,op_mac1_36b57b355b216dab_count,op_mac1_aca1e020465b752c_count,op_mac2_nunique,op_mac2_a8dc52f65085212e_count,op_mac2_b910ea1cd2e10a1d_count,op_mac2_e241b5cadbabf12a_count,op_mac2_25ad3858d08d4757_count,op_mac2_97343db8417d9d5b_count,op_ip1_nunique,op_ip1_0fe293bea342665a_count,op_ip1_99533c7d37963762_count,op_ip1_3503aab90a981cc6_count,op_ip1_211d0e54f3da2e8d_count,op_ip1_329ad082dc996703_count,op_ip2_nunique,op_ip2_ba85d4d236ef2c06_count,op_ip2_5777335f8759f4f4_count,op_ip2_1cff430529049618_count,op_ip2_b8cbb028b65e693d_count,op_ip2_43017d3ac955020e_count,op_wifi_nunique,op_wifi_6ec718bbccbaedd2_count,op_wifi_2109917457c421d3_count,op_wifi_2c0f832d5d75448c_count,op_wifi_24f567277be92597_count,op_wifi_7ccf71075ea9aabf_count,op_geo_code_nunique,op_geo_code_ws05_count,op_geo_code_wkpk_count,op_geo_code_wt5c_count,op_geo_code_wmuk_count,op_geo_code_ws4u_count,op_ip1_sub_nunique,op_ip1_sub_f5009e2c6c13c80c_count,op_ip1_sub_72309d88abb9b916_count,op_ip1_sub_ace49c4dd8733686_count,op_ip1_sub_d7833f8214e8019f_count,op_ip1_sub_d212bf8069071714_count,op_ip2_sub_nunique,op_ip2_sub_0a52fd4ad5c3aa32_count,op_ip2_sub_ee2a277f07c7ac81_count,op_ip2_sub_c778bc48328749b4_count,op_ip2_sub_7661c9041bbe683c_count,op_ip2_sub_2b14a2d1bdfffb8d_count,op_device1_aca4977fbe8741e1_device2_IPHONE 5S_count,op_device1_aca4977fbe8741e1_device2_IPHONE 5_count,op_device1_aca4977fbe8741e1_device2_IPHONE 5C_count,op_device1_dcc26c88f36170cc_device2_SM-G955F_count,op_device1_aca4977fbe8741e1_device2_IPHONE 6_count,op_device1_630a1adff2a87007_device2_HM NOTE 1S_count,op_device1_aca4977fbe8741e1_device2_IPHONE 6 PLUS_count,op_device1_630a1adff2a87007_device2_REDMI 4X_count,op_device1_8f970961c66d9ec6_device2_ZTE BA603_count,op_device1_2fec0cf493057746_device2_MI 6 _count,op_device1_630a1adff2a87007_mac1_a8dc52f65085212e_count,op_device1_d2cf44cec09806cc_mac1_a8dc52f65085212e_count,op_device1_a92470e4f878a3f4_mac1_b1f92d4324c1ee16_count,op_device1_09baf2f39bc3dc86_mac1_a8dc52f65085212e_count,op_device1_dcc26c88f36170cc_mac1_9251892b01b1a219_count,op_device1_630a1adff2a87007_mac1_36b57b355b216dab_count,op_device1_49dd36968dbfadda_mac1_a8dc52f65085212e_count,op_device1_d2cf44cec09806cc_mac1_aca1e020465b752c_count,op_device1_d2cf44cec09806cc_mac1_65ea75cf787a04c9_count,op_device1_d2cf44cec09806cc_mac1_91fed3b7fae0eb1f_count,op_device_code1_c50faf7eba139d67_device_code2_483ddc3a31666b8f_count,op_device_code1_15d09ccab1356c7b_device_code2_c3126d3ef88b84d0_count,op_de

In [114]:
###### 对测试集进行相同的预处理
# 处理op_test的数据
# 先查看操作箱单数据表
test=sub
op_columns=['UID']

# 计算用户在30天内的总操作数
op_columns.append('op_operation_count')
test=test.merge(op_test.groupby(['UID'])['day'].count().reset_index(),on='UID',how='left')
# 操作日期 day
# 用户在30天内的活跃天数
op_columns.append('op_day_nunique')
test=test.merge(op_test.groupby(['UID'])['day'].nunique().reset_index(),on='UID',how='left')
# 用户在周末的操作次数
op_columns.append('op_operation_weekend_count')
test=test.merge(op_test[(op_test.week_day==0)|(op_test.week_day==6)].groupby(['UID'])['day'].count().reset_index(),on='UID',how='left')

# 用户有多少不同的操作类型
op_columns.append('op_mode_nunique')
test=test.merge(op_test.groupby(['UID'])['mode'].nunique().reset_index(),on='UID',how='left')

# 排名前5的操作类型
# c8741ce15ceac2a4,d25caee90b27fa9b,79f8b86938180c3c,b501fa4fc58206b9,acfaded7e04e7ba0
mode_top5=['c8741ce15ceac2a4','d25caee90b27fa9b','79f8b86938180c3c','b501fa4fc58206b9','acfaded7e04e7ba0']
for mode in mode_top5:
    op_columns.append('op_mode_'+mode+'_count')
    test=test.merge(op_test[op_test['mode']==mode].groupby(['UID'])['mode'].count().reset_index(),on='UID',how='left')

# 操作success的数量/总操作数
op_columns.append('op_success_mean')
test=test.merge(op_test.groupby(['UID'])['success'].mean().reset_index(),on='UID',how='left')

# 排名前5的hour和羊毛党操作最多的hour
# 8,9,10,11,14,15,16
hour_top5=[8,9,10,11,14,15,16]
for hour in hour_top5:
    op_columns.append('op_hour_'+str(hour)+'_count')
    test=test.merge(op_test[op_test['hour']==hour].groupby(['UID'])['hour'].count().reset_index(),on='UID',how='left')

# 不同的操作系统数量
op_columns.append('op_os_nunique')
test=test.merge(op_test.groupby(['UID'])['os'].nunique().reset_index(),on='UID',how='left')
# 排名前3的操作系统
os_top3=['102','200','103']
for os in os_top3:
    op_columns.append('op_os_'+os+'_count')
    test=test.merge(op_test[op_test['os']==os].groupby(['UID'])['os'].count().reset_index(),on='UID',how='left')

# 不同version的操作数量
op_columns.append('op_version_nunique')
test=test.merge(op_test.groupby(['UID'])['version'].nunique().reset_index(),on='UID',how='left')
# 排名前5的version
version_top5=['7.0.9','7.0.5','7.0.2','7.0.7','6.6.2','7.0.0']
for version in version_top5:
    op_columns.append('op_version_'+version+'_count')
    test=test.merge(op_test[op_test['version']==version].groupby(['UID'])['version'].count().reset_index(),on='UID',how='left')

# device1
op_columns.append('op_device1_nunique')
test=test.merge(op_test.groupby(['UID'])['device1'].nunique().reset_index(),on='UID',how='left')
device1_UID_num=op_train[['UID','device1']].drop_duplicates().groupby(['device1'])['UID'].count().sort_values(ascending=False)[0:10].index.tolist()
device1_top5=['09baf2f39bc3dc86', 'aca4977fbe8741e1', 'd2cf44cec09806cc', '49dd36968dbfadda', '8399be6b80847240', '630a1adff2a87007']
device1_top=set(device1_UID_num)|set(device1_top5)
for device1 in device1_top:
    op_columns.append('op_device1_'+device1+'_count')
    test=test.merge(op_train[op_test['device1']==device1].groupby(['UID'])['device1'].count().reset_index(),on='UID',how='left')

# 计算属性值和TopN的计数值
cols=['device_code2', 'device_code3', 'mac1','mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub']
for col in cols:
    op_columns.append('op_'+col+'_nunique')
    test=test.merge(op_test.groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
    index1=op_train.groupby([col])['UID'].count().sort_values(ascending=False)[0:5].index
    index2=op_train_y[op_train_y.Tag==1][col].value_counts()[0:5].index
    indexs=set(index1)|set(index2)
    
#     UID_num=op_train[['UID',col]].drop_duplicates().groupby([col])['UID'].count().sort_values(ascending=False)[0:10].index.tolist()
#     indexs=set(indexs)|set(UID_num)
    for index in index2:
        op_columns.append('op_'+col+'_'+index+'_count')
        test=test.merge(op_test[op_test[col]==index].groupby(['UID'])[col].count().reset_index(),on='UID',how='left')

# # longtitude
# col='longtitude'
# op_columns.append('op_'+col+'_count')
# op_columns.append('op_'+col+'_nunique')
# op_columns.append('op_'+col+'_max')
# op_columns.append('op_'+col+'_min')
# op_columns.append('op_'+col+'_sum')
# op_columns.append('op_'+col+'_mean')
# op_columns.append('op_'+col+'_median')
# op_columns.append('op_'+col+'_std')
# op_columns.append('op_'+col+'_skew')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].max().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].min().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].sum().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].mean().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].median().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].std().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].skew().reset_index(),on='UID',how='left')

# # latitude
# col='latitude'
# op_columns.append('op_'+col+'_count')
# op_columns.append('op_'+col+'_nunique')
# op_columns.append('op_'+col+'_max')
# op_columns.append('op_'+col+'_min')
# op_columns.append('op_'+col+'_sum')
# op_columns.append('op_'+col+'_mean')
# op_columns.append('op_'+col+'_median')
# op_columns.append('op_'+col+'_std')
# op_columns.append('op_'+col+'_skew')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].max().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].min().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].sum().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].mean().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].median().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].std().reset_index(),on='UID',how='left')
# test =test.merge(op_test[op_test[col]!=0].groupby(['UID'])[col].skew().reset_index(),on='UID',how='left')

# 二元组合变量
bivariable=[['device1','device2'],['device1','mac1'],['device_code1','device_code2'],['device_code1','geo_code'],['device_code1','ip1']]
for var in bivariable:
    var1=var[0]
    var2=var[1]
    biparam=op_train_y[op_train_y.Tag==1].groupby([var1,var2])['UID'].count().sort_values(ascending=False).reset_index()[0:10]
    for param1,param2 in zip(biparam[var1],biparam[var2]):
        op_columns.append('op_'+var1+'_'+str(param1)+'_'+var2+'_'+str(param2)+'_count')
        test=test.merge(op_test[(op_test[var1]==param1)&(op_test[var2]==param2)].groupby(['UID']).size().reset_index(),on='UID',how='left')  

In [115]:
op_test_int_time=op_test.groupby(['UID','day'])['int_time'].std().reset_index()
op_test_int_time_std_min=op_test_int_time[op_test_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].min().reset_index()
op_test_int_time_std_max=op_test_int_time[op_test_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].max().reset_index()
op_columns.append('op_'+'int_time'+'_std_min')
op_columns.append('op_'+'int_time'+'_std_max')
op_columns.append('op_'+'int_time'+'_std_mean')
op_columns.append('op_'+'int_time'+'_std_sum')
test=test.merge(op_test_int_time_std_min,on='UID',how='left')
test=test.merge(op_test_int_time_std_max,on='UID',how='left')
test=test.merge(op_test_int_time[op_test_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].mean().reset_index(),on='UID',how='left')
test=test.merge(op_test_int_time[op_test_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].sum().reset_index(),on='UID',how='left')

In [116]:
# geo_code_list=op_test[['UID','longtitude','latitude']].dropna().drop_duplicates()
# UID_list=geo_code_list.UID.drop_duplicates()
# df = pd.DataFrame()
# for uid in UID_list:
#     longtitude=geo_code_list[geo_code_list.UID==uid]['longtitude'].tolist()
#     latitude=geo_code_list[geo_code_list.UID==uid]['latitude'].tolist()
#     count=len(longtitude)
#     dis_mean=0
#     dis_list=[]
#     if count==1:
#         dis_mean=0
#     else:
#         for i in range(count-1):
#             lo=longtitude[i]
#             la=latitude[i]
#             #print(lo,la)
#             dis=[calcDistance(la,lo,latitude[j],longtitude[j]) for j in range(i+1,count)]
#             dis_list.extend(dis)
#         dis_mean=np.mean(dis_list)
#         #print(dis_mean)
#     df2=pd.DataFrame({'UID':uid, 'distance_mean':dis_mean},index=["0"])
#     df=df.append(df2, ignore_index=True)
# op_columns.append('op_geo_code_distance_mean')
# test=test.merge(df,on='UID',how='left')

In [117]:
# 处理trans_test的数据
# 计算用户在30天内的总操作数
op_columns.append('trans_operation_count')
test=test.merge(trans_test.groupby(['UID'])['day'].count().reset_index(),on='UID',how='left')

# 用户在周末的操作次数
op_columns.append('trans_operation_weekend_count')
test=test.merge(trans_test[(trans_test.week_day==0)|(trans_test.week_day==6)].groupby(['UID'])['day'].count().reset_index(),on='UID',how='left')


# 计算属性值和TopN的计数值
cols=['channel', 'day', 'amt_src1', 'merchant',
       'code1', 'code2', 'trans_type1', 'acc_id1', 'device_code1',
       'device_code2', 'device_code3', 'device1', 'device2', 'mac1', 'ip1',
       'amt_src2', 'acc_id2', 'acc_id3', 'geo_code', 'trans_type2',
       'market_code', 'market_type', 'ip1_sub', 'hour']
for col in cols:
    op_columns.append('trans_'+str(col)+'_nunique')
    test=test.merge(trans_test.groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
    index1=trans_train.groupby([col])['UID'].count().sort_values(ascending=False)[0:5].index
    index2=trans_train_y[trans_train_y.Tag==1][col].value_counts()[0:5].index
    indexs=set(index1)|set(index2)
    for index in index2:
        op_columns.append('trans_'+str(col)+'_'+str(index)+'_count')
        test=test.merge(trans_test[trans_test[col]==index].groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
        
# trans_amt_rank_discret/bal_rank_discret
# rank=[1,2,3,4,5,6,7,8,9,10]
# for r in rank:
#     op_columns.append('trans_trans_amt_rank_discret'+'_'+str(r)+'_count')
#     test=test.merge(trans_test[trans_test['trans_amt_rank_discret']==r].groupby(['UID'])['trans_amt_rank_discret'].count().reset_index(),on='UID',how='left')

# for r in rank:
#     op_columns.append('trans_bal_rank_discret'+'_'+str(r)+'_count')
#     test=test.merge(trans_test[trans_test['bal_rank_discret']==r].groupby(['UID'])['bal_rank_discret'].count().reset_index(),on='UID',how='left')

col='trans_amt'
op_columns.append('trans_'+col+'_count')
op_columns.append('trans_'+col+'_nunique')
op_columns.append('trans_'+col+'_max')
op_columns.append('trans_'+col+'_min')
op_columns.append('trans_'+col+'_sum')
op_columns.append('trans_'+col+'_mean')
op_columns.append('trans_'+col+'_median')
op_columns.append('trans_'+col+'_std')
op_columns.append('trans_'+col+'_skew')
test =test.merge(trans_test.groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].max().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].min().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].sum().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].mean().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].median().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].std().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].skew().reset_index(),on='UID',how='left')

col='bal'
op_columns.append('trans_'+col+'_count')
op_columns.append('trans_'+col+'_nunique')
op_columns.append('trans_'+col+'_max')
op_columns.append('trans_'+col+'_min')
op_columns.append('trans_'+col+'_sum')
op_columns.append('trans_'+col+'_mean')
op_columns.append('trans_'+col+'_median')
op_columns.append('trans_'+col+'_std')
op_columns.append('trans_'+col+'_skew')
test =test.merge(trans_test.groupby(['UID'])[col].count().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].nunique().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].max().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].min().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].sum().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].mean().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].median().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].std().reset_index(),on='UID',how='left')
test =test.merge(trans_test.groupby(['UID'])[col].skew().reset_index(),on='UID',how='left')

# 处理交易金额
op_columns.append('trans_'+'log_trans_amt_lower_5'+'_count')
test=test.merge(trans_test[trans_test['log_trans_amt']<5].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')  

op_columns.append('trans_'+'log_trans_amt_5_to_6'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=5)&(trans_test['log_trans_amt']<6)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_6_to_7'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=6)&(trans_test['log_trans_amt']<7)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_7_to_8'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=7)&(trans_test['log_trans_amt']<8)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_8_to_9'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=8)&(trans_test['log_trans_amt']<9)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_9_to_10'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=9)&(trans_test['log_trans_amt']<10)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_10_to_11'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=10)&(trans_test['log_trans_amt']<11)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_11_to_12'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=11)&(trans_test['log_trans_amt']<12)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_12_to_13'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=12)&(trans_test['log_trans_amt']<13)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_13_to_14'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=13)&(trans_test['log_trans_amt']<14)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_14_to_15'+'_count')
test=test.merge(trans_test[(trans_test['log_trans_amt']>=14)&(trans_test['log_trans_amt']<=15)].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

op_columns.append('trans_'+'log_trans_amt_greater_15'+'_count')
test=test.merge(trans_test[trans_test['log_trans_amt']>15].groupby(['UID']).size().reset_index().fillna(0),on='UID',how='left')

In [118]:
# geo_code_list=trans_test[['UID','longtitude','latitude']].dropna().drop_duplicates()
# UID_list=geo_code_list.UID.drop_duplicates()
# df = pd.DataFrame()
# for uid in UID_list:
#     longtitude=geo_code_list[geo_code_list.UID==uid]['longtitude'].tolist()
#     latitude=geo_code_list[geo_code_list.UID==uid]['latitude'].tolist()
#     count=len(longtitude)
#     dis_mean=0
#     dis_list=[]
#     if count==1:
#         dis_mean=0
#     else:
#         for i in range(count-1):
#             lo=longtitude[i]
#             la=latitude[i]
#             #print(lo,la)
#             dis=[calcDistance(la,lo,latitude[j],longtitude[j]) for j in range(i+1,count)]
#             dis_list.extend(dis)
#         dis_mean=np.mean(dis_list)
#         #print(dis_mean)
#     df2=pd.DataFrame({'UID':uid, 'distance_mean':dis_mean},index=["0"])
#     df=df.append(df2, ignore_index=True)
# op_columns.append('trans_geo_code_distance_mean')
# test=test.merge(df,on='UID',how='left')

In [119]:
trans_test_int_time=trans_test.groupby(['UID','day'])['int_time'].std().reset_index()
trans_test_int_time_std_min=trans_test_int_time[trans_test_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].min().reset_index()
trans_test_int_time_std_max=trans_test_int_time[trans_test_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].max().reset_index()
op_columns.append('trans_'+'int_time'+'_std_min')
op_columns.append('trans_'+'int_time'+'_std_max')
op_columns.append('trans_'+'int_time'+'_std_mean')
op_columns.append('trans_'+'int_time'+'_std_sum')
test=test.merge(trans_test_int_time_std_min,on='UID',how='left')
test=test.merge(trans_test_int_time_std_max,on='UID',how='left')
test=test.merge(trans_test_int_time[trans_test_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].mean().reset_index(),on='UID',how='left')
test=test.merge(trans_test_int_time[trans_test_int_time.int_time!=0].dropna().groupby(['UID'])['int_time'].sum().reset_index(),on='UID',how='left')

In [120]:
# 修改列名
test.columns=op_columns

# 用户操作总数/活跃天数
test['op_operation_per_day']=test['op_operation_count']/test['op_day_nunique']
# 用户周末操作数/总操作数
test['op_operation_per_weekend']=test['op_operation_weekend_count']/test['op_operation_count']

# 用户交易总数/活跃天数
test['trans_operation_per_day']=test['trans_operation_count']/test['trans_day_nunique']
# 用户周末交易数/总交易数
test['trans_operation_per_weekend']=test['trans_operation_weekend_count']/test['trans_operation_count']

#用户交易数/操作数
test['trans_per_op']=test['trans_operation_count']/test['op_operation_count']
# 用户操作数/交易数
test['op_per_trans']=test['op_operation_count']/test['trans_operation_count']

# 交易金额小于5的操作百分比
test['trans_log_trans_amt_lower_5_percent']=test['trans_log_trans_amt_lower_5_count']/test['trans_operation_count']

# 交易金额5-6的操作百分比
test['trans_log_trans_amt_5_to_6_percent']=test['trans_log_trans_amt_5_to_6_count']/test['trans_operation_count']

# 交易金额6-7的操作百分比
test['trans_log_trans_amt_6_to_7_percent']=test['trans_log_trans_amt_6_to_7_count']/test['trans_operation_count']
test['trans_log_trans_amt_7_to_8_percent']=test['trans_log_trans_amt_7_to_8_count']/test['trans_operation_count']
test['trans_log_trans_amt_8_to_9_percent']=test['trans_log_trans_amt_8_to_9_count']/test['trans_operation_count']
test['trans_log_trans_amt_9_to_10_percent']=test['trans_log_trans_amt_9_to_10_count']/test['trans_operation_count']
test['trans_log_trans_amt_10_to_11_percent']=test['trans_log_trans_amt_10_to_11_count']/test['trans_operation_count']
test['trans_log_trans_amt_11_to_12_percent']=test['trans_log_trans_amt_11_to_12_count']/test['trans_operation_count']
test['trans_log_trans_amt_12_to_13_percent']=test['trans_log_trans_amt_12_to_13_count']/test['trans_operation_count']
test['trans_log_trans_amt_13_to_14_percent']=test['trans_log_trans_amt_13_to_14_count']/test['trans_operation_count']
test['trans_log_trans_amt_14_to_15_percent']=test['trans_log_trans_amt_14_to_15_count']/test['trans_operation_count']

# 交易金额大于15的操作百分比
test['trans_log_trans_amt_greater_15_percent']=test['trans_log_trans_amt_greater_15_count']/test['trans_operation_count']

In [121]:
pd.set_option('display.max_columns', None)
test.head()

,UID,op_operation_count,op_day_nunique,op_operation_weekend_count,op_mode_nunique,op_mode_c8741ce15ceac2a4_count,op_mode_d25caee90b27fa9b_count,op_mode_79f8b86938180c3c_count,op_mode_b501fa4fc58206b9_count,op_mode_acfaded7e04e7ba0_count,op_success_mean,op_hour_8_count,op_hour_9_count,op_hour_10_count,op_hour_11_count,op_hour_14_count,op_hour_15_count,op_hour_16_count,op_os_nunique,op_os_102_count,op_os_200_count,op_os_103_count,op_version_nunique,op_version_7.0.9_count,op_version_7.0.5_count,op_version_7.0.2_count,op_version_7.0.7_count,op_version_6.6.2_count,op_version_7.0.0_count,op_device1_nunique,op_device1_630a1adff2a87007_count,op_device1_49dd36968dbfadda_count,op_device1_09baf2f39bc3dc86_count,op_device1_dcc26c88f36170cc_count,op_device1_8399be6b80847240_count,op_device1_0d4e77d48255a0a2_count,op_device1_bf872c181a38310a_count,op_device1_a92470e4f878a3f4_count,op_device1_aca4977fbe8741e1_count,op_device1_8f970961c66d9ec6_count,op_device1_d2cf44cec09806cc_count,op_device_code2_nunique,op_device_code2_483ddc3a31666b8f_count,op_device_code2_c3126d3ef88b84d0_count,op_device_code2_09b787daa79cead5_count,op_device_code2_59dcb10df09acd30_count,op_device_code2_83cd44a2841ceecc_count,op_device_code3_nunique,op_device_code3_42100099fcace21b_count,op_device_code3_a2ae68874761cf91_count,op_device_code3_0d1884730db15ddc_count,op_device_code3_d059c94c8b9fbfb9_count,op_device_code3_27fe4b8e9696e8ae_count,op_mac1_nunique,op_mac1_a8dc52f65085212e_count,op_mac1_b1f92d4324c1ee16_count,op_mac1_9251892b01b1a219_count,op_mac1_36b57b355b216dab_count,op_mac1_aca1e020465b752c_count,op_mac2_nunique,op_mac2_a8dc52f65085212e_count,op_mac2_b910ea1cd2e10a1d_count,op_mac2_e241b5cadbabf12a_count,op_mac2_25ad3858d08d4757_count,op_mac2_97343db8417d9d5b_count,op_ip1_nunique,op_ip1_0fe293bea342665a_count,op_ip1_99533c7d37963762_count,op_ip1_3503aab90a981cc6_count,op_ip1_211d0e54f3da2e8d_count,op_ip1_329ad082dc996703_count,op_ip2_nunique,op_ip2_ba85d4d236ef2c06_count,op_ip2_5777335f8759f4f4_count,op_ip2_1cff430529049618_count,op_ip2_b8cbb028b65e693d_count,op_ip2_43017d3ac955020e_count,op_wifi_nunique,op_wifi_6ec718bbccbaedd2_count,op_wifi_2109917457c421d3_count,op_wifi_2c0f832d5d75448c_count,op_wifi_24f567277be92597_count,op_wifi_7ccf71075ea9aabf_count,op_geo_code_nunique,op_geo_code_ws05_count,op_geo_code_wkpk_count,op_geo_code_wt5c_count,op_geo_code_wmuk_count,op_geo_code_ws4u_count,op_ip1_sub_nunique,op_ip1_sub_f5009e2c6c13c80c_count,op_ip1_sub_72309d88abb9b916_count,op_ip1_sub_ace49c4dd8733686_count,op_ip1_sub_d7833f8214e8019f_count,op_ip1_sub_d212bf8069071714_count,op_ip2_sub_nunique,op_ip2_sub_0a52fd4ad5c3aa32_count,op_ip2_sub_ee2a277f07c7ac81_count,op_ip2_sub_c778bc48328749b4_count,op_ip2_sub_7661c9041bbe683c_count,op_ip2_sub_2b14a2d1bdfffb8d_count,op_device1_aca4977fbe8741e1_device2_IPHONE 5S_count,op_device1_aca4977fbe8741e1_device2_IPHONE 5_count,op_device1_aca4977fbe8741e1_device2_IPHONE 5C_count,op_device1_dcc26c88f36170cc_device2_SM-G955F_count,op_device1_aca4977fbe8741e1_device2_IPHONE 6_count,op_device1_630a1adff2a87007_device2_HM NOTE 1S_count,op_device1_aca4977fbe8741e1_device2_IPHONE 6 PLUS_count,op_device1_630a1adff2a87007_device2_REDMI 4X_count,op_device1_8f970961c66d9ec6_device2_ZTE BA603_count,op_device1_2fec0cf493057746_device2_MI 6 _count,op_device1_630a1adff2a87007_mac1_a8dc52f65085212e_count,op_device1_d2cf44cec09806cc_mac1_a8dc52f65085212e_count,op_device1_a92470e4f878a3f4_mac1_b1f92d4324c1ee16_count,op_device1_09baf2f39bc3dc86_mac1_a8dc52f65085212e_count,op_device1_dcc26c88f36170cc_mac1_9251892b01b1a219_count,op_device1_630a1adff2a87007_mac1_36b57b355b216dab_count,op_device1_49dd36968dbfadda_mac1_a8dc52f65085212e_count,op_device1_d2cf44cec09806cc_mac1_aca1e020465b752c_count,op_device1_d2cf44cec09806cc_mac1_65ea75cf787a04c9_count,op_device1_d2cf44cec09806cc_mac1_91fed3b7fae0eb1f_count,op_device_code1_c50faf7eba139d67_device_code2_483ddc3a31666b8f_count,op_device_code1_15d09ccab1356c7b_device_code2_c3126d3ef88b84d0_count,op_device

In [122]:
test=test.fillna(-1)
train=train.fillna(-1)

train.trans_per_op[np.isinf(train.trans_per_op)]=train.trans_per_op.mean()
test.trans_per_op[np.isinf(test.trans_per_op)]=test.trans_per_op.mean()

train.op_per_trans[np.isinf(train.op_per_trans)]=train.op_per_trans.mean()
test.op_per_trans[np.isinf(test.op_per_trans)]=test.op_per_trans.mean()

# 保存数据
train.to_csv('data/feature/train.csv',index=False)
test.to_csv('data/feature/test.csv',index=False)

In [123]:
target=train.Tag
feature=train.drop(['UID','Tag'],axis=1)

UID=test.UID
test_feature=test.drop(['UID'],axis=1)

### Xgboost

In [ ]:
# 训练
xgb_c = XGBClassifier(learning_rate =0.1,n_estimators=1000,max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,
                     objective= 'binary:logistic',nthread=4,scale_pos_weight=26894/4285,seed=27)

xgb_param = xgb_c.get_xgb_params()
xgtrain = xgb.DMatrix(feature.values, label=target.values)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=xgb_c.get_params()['n_estimators'], nfold=5, metrics='auc', early_stopping_rounds=50)
xgb_c.set_params(n_estimators=cvresult.shape[0])
xgb_c.fit(feature, target,eval_metric='auc')
predprob = xgb_c.predict_proba(test_feature)[:,1]

In [ ]:
test_y=pd.Series(predprob,name='Tag')
test_data=pd.concat([UID,test_y],axis=1)

In [ ]:
test_data.to_csv('data/test_predict.csv',index=0,float_format='%.6f')

### Lightgbm

In [124]:
#官方的评价指标
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 'TC_AUC',0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3,True

In [125]:
lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=64, reg_alpha=0, reg_lambda=0, max_depth=-1,
    n_estimators=1000, objective='binary', subsample=0.9, colsample_bytree=0.8, subsample_freq=1, learning_rate=0.05,
    random_state=1024, n_jobs=10, min_child_weight=4, min_child_samples=5, min_split_gain=0, silent=True,scale_pos_weight=26894/4285)


skf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)
best_score = []

oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.UID.shape[0])


for index, (train_index, test_index) in enumerate(skf.split(feature, target)):
    lgb_model.fit(feature.iloc[train_index], target.iloc[train_index], verbose=50,
                  eval_set=[(feature.iloc[train_index], target.iloc[train_index]),
                            (feature.iloc[test_index], target.iloc[test_index])], early_stopping_rounds=100)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    oof_preds[test_index] = lgb_model.predict_proba(feature.iloc[test_index], num_iteration=lgb_model.best_iteration_)[:,1]

    test_pred = lgb_model.predict_proba(test_feature, num_iteration=lgb_model.best_iteration_)[:, 1]
    sub_preds += test_pred / 5

score = tpr_weight_funtion(y_predict=oof_preds,y_true=target)
print('score:',score[1])

# test_y=pd.Series(sub_preds,name='Tag')
# test_data=pd.concat([UID,test_y],axis=1)

# test_data.to_csv('data/test_predict.csv',index=0,float_format='%.6f')

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.141216	valid_1's binary_logloss: 0.16775
[100]	valid_0's binary_logloss: 0.0901241	valid_1's binary_logloss: 0.133292
[150]	valid_0's binary_logloss: 0.0620933	valid_1's binary_logloss: 0.11882
[200]	valid_0's binary_logloss: 0.0432279	valid_1's binary_logloss: 0.110344
[250]	valid_0's binary_logloss: 0.0307662	valid_1's binary_logloss: 0.106471
[300]	valid_0's binary_logloss: 0.0222233	valid_1's binary_logloss: 0.105349
[350]	valid_0's binary_logloss: 0.016216	valid_1's binary_logloss: 0.105871
Early stopping, best iteration is:
[297]	valid_0's binary_logloss: 0.0226386	valid_1's binary_logloss: 0.105115
[0.10511465363833179]
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.14088	valid_1's binary_logloss: 0.169331
[100]	valid_0's binary_logloss: 0.0898928	valid_1's binary_logloss: 0.135811
[150]	valid_0's binary_logloss: 0.0612931	valid_1's bin

In [126]:
#特征重要性
se = pd.Series(lgb_model.feature_importances_,index=feature.columns)
col =se.sort_values(ascending=False)
col.to_csv(path + '/col_sort_one.csv')

In [127]:
# 删除不重要特征以后，再重新训练
nonimportance_f=col[col<=10].index.tolist()
feature2=feature.drop(nonimportance_f,axis=1)
test_feature2=test_feature.drop(nonimportance_f,axis=1)

lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=64, reg_alpha=0, reg_lambda=0, max_depth=-1,
    n_estimators=1000, objective='binary', subsample=0.9, colsample_bytree=0.8, subsample_freq=1, learning_rate=0.05,
    random_state=1024, n_jobs=10, min_child_weight=4, min_child_samples=5, min_split_gain=0, silent=True,scale_pos_weight=26894/4285)


skf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)
best_score = []

oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.UID.shape[0])


for index, (train_index, test_index) in enumerate(skf.split(feature2, target)):
    lgb_model.fit(feature2.iloc[train_index], target.iloc[train_index], verbose=50,
                  eval_set=[(feature2.iloc[train_index], target.iloc[train_index]),
                            (feature2.iloc[test_index], target.iloc[test_index])], early_stopping_rounds=100)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    oof_preds[test_index] = lgb_model.predict_proba(feature2.iloc[test_index], num_iteration=lgb_model.best_iteration_)[:,1]

    test_pred = lgb_model.predict_proba(test_feature2, num_iteration=lgb_model.best_iteration_)[:, 1]
    sub_preds += test_pred / 5

score = tpr_weight_funtion(y_predict=oof_preds,y_true=target)
print('score:',score[1])

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.141996	valid_1's binary_logloss: 0.169664
[100]	valid_0's binary_logloss: 0.0898424	valid_1's binary_logloss: 0.135717
[150]	valid_0's binary_logloss: 0.0616013	valid_1's binary_logloss: 0.120589
[200]	valid_0's binary_logloss: 0.04291	valid_1's binary_logloss: 0.112385
[250]	valid_0's binary_logloss: 0.0306465	valid_1's binary_logloss: 0.108731
[300]	valid_0's binary_logloss: 0.022005	valid_1's binary_logloss: 0.10772
[350]	valid_0's binary_logloss: 0.0161048	valid_1's binary_logloss: 0.108289
[400]	valid_0's binary_logloss: 0.0120966	valid_1's binary_logloss: 0.109959
Early stopping, best iteration is:
[312]	valid_0's binary_logloss: 0.020345	valid_1's binary_logloss: 0.107611
[0.10761109804439611]
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's binary_logloss: 0.14167	valid_1's binary_logloss: 0.169985
[100]	valid_0's binary_logloss: 0.0911431	valid_1's binar

### 规则调整

In [153]:
rule_code = ['5776870b5747e14e' ,'8b3f74a1391b5427' ,'0e90f47392008def' ,'6d55ccc689b910ee' ,'2260d61b622795fb' ,'1f72814f76a984fa' ,'c2e87787a76836e0' ,'4bca6018239c6201' ,'922720f3827ccef8' ,'2b2e7046145d9517' ,'09f911b8dc5dfc32' ,'7cc961258f4dce9c' ,'bc0213f01c5023ac' ,'0316dca8cc63cc17' ,'c988e79f00cc2dc0' ,'d0b1218bae116267' ,'72fac912326004ee' ,'00159b7cc2f1dfc8' ,'49ec5883ba0c1b0e' ,'c9c29fc3d44a1d7b' ,'33ce9c3877281764' ,'e7c929127cdefadb' ,'05bc3e22c112c8c9' ,'5cf4f55246093ccf' ,'6704d8d8d5965303' ,'4df1708c5827264d' ,'6e8b399ffe2d1e80' ,'f65104453e0b1d10' ,'1733ddb502eb3923' ,'a086f47f681ad851' ,'1d4372ca8a38cd1f' ,'29db08e2284ea103' ,'4e286438d39a6bd4' ,'54cb3985d0380ca4' ,'6b64437be7590eb0' ,'89eb97474a6cb3c6' ,'95d506c0e49a492c' ,'c17b47056178e2bb' ,'d36b25a74285bebb']
test_rule_uid = pd.DataFrame(trans_test[trans_test['merchant'].isin(rule_code)].UID.unique())
merchant_rule=test_rule_uid[0].tolist()
test_y=pd.Series(sub_preds,name='Tag')
test_data=pd.concat([UID,test_y],axis=1)
test_data['Tag'].loc[test_data[test_data['UID'].isin(merchant_rule)].index]=1

In [155]:
test_data.to_csv('data/test_predict.csv',index=0,float_format='%.6f')